# Horovod Distributed Training with SageMaker TensorFlow script mode.

Horovod is a distributed training framework based on Message Passing Interfae (MPI). For information about Horovod, see [Horovod README](https://github.com/uber/horovod).

You can perform distributed training with Horovod on SageMaker by using the SageMaker Tensorflow container. If MPI is enabled when you create the training job, SageMaker creates the MPI environment and executes the `mpirun` command to execute the training script. Details on how to configure mpi settings in training job are described later in this example.

In this example notebook, we create a Horovod training job that uses the MNIST data set.

## Set up the environment

We get the `IAM` role that this notebook is running as and pass that role to the TensorFlow estimator that SageMaker uses to get data and perform training.

In [159]:
import sagemaker
import os
from sagemaker.utils import sagemaker_timestamp
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role
import time

sagemaker_session = sagemaker.Session()

default_s3_bucket = sagemaker_session.default_bucket()
sagemaker_iam_role = get_execution_role()

train_script = "mnist_hvd.py"
instance_count = 2

## Prepare Data for training

Now we download the MNIST dataset to the local `/tmp/data/` directory and then upload it to an S3 bucket. After uploading the dataset to S3, we delete the data from `/tmp/data/`. 

In [131]:
import os
import shutil

import numpy as np

import keras
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

s3_train_path = "s3://{}/mnist/train.npz".format(default_s3_bucket)
s3_test_path = "s3://{}/mnist/test.npz".format(default_s3_bucket)

# Create local directory
! mkdir -p /tmp/data/mnist_train
! mkdir -p /tmp/data/mnist_test

# Save data locally
np.savez('/tmp/data/mnist_train/train.npz', data=x_train, labels=y_train)
np.savez('/tmp/data/mnist_test/test.npz', data=x_test, labels=y_test)

# Upload the dataset to s3
! aws s3 cp /tmp/data/mnist_train/train.npz $s3_train_path
! aws s3 cp /tmp/data/mnist_test/test.npz $s3_test_path

print('training data at ', s3_train_path)
print('test data at ', s3_test_path)
! rm -rf /tmp/data

upload: ../../../../tmp/data/mnist_train/train.npz to s3://sagemaker-us-east-1-328296961357/mnist/train.npz
upload: ../../../../tmp/data/mnist_test/test.npz to s3://sagemaker-us-east-1-328296961357/mnist/test.npz
training data at  s3://sagemaker-us-east-1-328296961357/mnist/train.npz
test data at  s3://sagemaker-us-east-1-328296961357/mnist/test.npz


## Write a script for horovod distributed training

This example is based on the [Keras MNIST horovod example](https://github.com/uber/horovod/blob/master/examples/keras_mnist.py) example in the horovod github repository.

To run this script we have to make following modifications:

### 1. Accept `--model_dir` as a command-line argument
Modify the script to accept `model_dir` as a command-line argument that defines the directory path (i.e. `/opt/ml/model/`) where the output model is saved. Because Sagemaker deletes the training cluster when training completes, saving the model to `/opt/ml/model/` directory prevents the trained model from getting lost, because when the training job completes, SageMaker writes the data stored in `/opt/ml/model/` to an S3 bucket. 

This also allows the SageMaker training job to integrate with other SageMaker services, such as hosted inference endpoints or batch transform jobs. It also allows you to host the trained model outside of SageMaker.

The following code adds `model_dir` as a command-line argument to the script:

```
parser = argparse.ArgumentParser()
parser.add_argument('--model_dir', type=str)
```

More details can be found [here](https://github.com/aws/sagemaker-containers/blob/master/README.md).

### 2. Load train and test data

You can get local directory path where the `train` and `test` data is downloaded by reading the environment variable `SM_CHANNEL_TRAIN` and `SM_CHANNEL_TEST` respectively.
After you get the directory path, load the data into memory.

Here is the code:

```
x_train = np.load(os.path.join(os.environ['SM_CHANNEL_TRAIN'], 'train.npz'))['data']
y_train = np.load(os.path.join(os.environ['SM_CHANNEL_TRAIN'], 'train.npz'))['labels']

x_test = np.load(os.path.join(os.environ['SM_CHANNEL_TEST'], 'test.npz'))['data']
y_test = np.load(os.path.join(os.environ['SM_CHANNEL_TEST'], 'test.npz'))['labels']
```

For a list of all environment variables set by SageMaker that are accessible inside a training script, see [SageMaker Containers](https://github.com/aws/sagemaker-containers/blob/master/README.md).

### 3. Save the model only at the master node

Because in Horovod the training is distributed to multiple nodes, the model should only be saved by the master node. The following code in the script does this:

```
# Horovod: Save model only on worker 0 (i.e. master)
if hvd.rank() == 0:
    saved_model_path = tf.contrib.saved_model.save_keras_model(model, args.model_dir)
```

### Training script

Here is the final training script.

In [132]:
!cat 'mnist_hvd.py'

#     Copyright 2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#     Licensed under the Apache License, Version 2.0 (the "License").
#     You may not use this file except in compliance with the License.
#     A copy of the License is located at
#
#         https://aws.amazon.com/apache-2-0/
#
#     or in the "license" file accompanying this file. This file is distributed
#     on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
#     express or implied. See the License for the specific language governing
#     permissions and limitations under the License.

from __future__ import print_function

import argparse
import os

import numpy as np

# import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import math
import tensor

## Test locally using SageMaker Python SDK TensorFlow Estimator

You can use the SageMaker Python SDK TensorFlow estimator to easily train locally and in SageMaker.

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments. Just change your estimator's `train_instance_type` to `local` or `local_gpu`. For more information, see: https://github.com/aws/sagemaker-python-sdk#local-mode.

To use this feature, you need to install docker-compose (and nvidia-docker if you are training with a GPU). Run the following script to install docker-compose or nvidia-docker-compose, and configure the notebook environment for you.

**Note**: You can only run a single local notebook at a time.

In [133]:
!/bin/bash ./setup.sh

/bin/bash: ./setup.sh: No such file or directory


To train locally, set `train_instance_type` to `local`:

In [134]:
train_instance_type='local'

The MPI environment for Horovod can be configured by setting the following flags in the `mpi` field of the `distribution` dictionary that you pass to the TensorFlow estimator :

* ``enabled (bool)``: If set to ``True``, the MPI setup is performed and ``mpirun`` command is executed.
* ``processes_per_host (int) [Optional]``: Number of processes MPI should launch on each host. Note, this should not be greater than the available slots on the selected instance type. This flag should be set for the multi-cpu/gpu training.
* ``custom_mpi_options (str) [Optional]``: Any mpirun flag(s) can be passed in this field that will be added to the mpirun command executed by SageMaker to launch distributed horovod training.

For more information about the `distribution` dictionary, see the SageMaker Python SDK [README](https://github.com/aws/sagemaker-python-sdk/blob/v1.17.3/src/sagemaker/tensorflow/README.rst).

First, enable MPI:

In [135]:
distributions = {'mpi': {'enabled': True}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution`

In [136]:
estimator_local = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       base_job_name='hvd-mnist-local')

tensorflow py2 container will be deprecated soon.


Call `fit()` to start the local training 

In [137]:
#%%time
#estimator_local.fit({"train":s3_train_path, "test":s3_test_path})

## Train in SageMaker

After you test the training job locally, run it on SageMaker:

First, change the instance type from `local` to the valid EC2 instance type. For example, `ml.c4.xlarge`.

In [138]:
#train_instance_type='ml.c4.xlarge' #4  vCPU
train_instance_type='ml.c4.4xlarge' #16 vCPU

You can also provide your custom MPI options by passing in the `custom_mpi_options` field of `distribution` dictionary that will be added to the `mpirun` command executed by SageMaker:

In [139]:
distributions = {'mpi': {'enabled': True, "custom_mpi_options": "-verbose --NCCL_DEBUG=INFO"}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution` to launch the training job in sagemaker.

In [140]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       base_job_name='hvd-mnist')

tensorflow py2 container will be deprecated soon.


Call `fit()` to start the training

In [141]:
#%%time
#estimator.fit({"train":s3_train_path, "test":s3_test_path})

##  Horovod training in SageMaker using multiple CPU/GPU

To enable mulitiple CPUs or GPUs for horovod training, set the `processes_per_host` field in the `mpi` section of the `distribution` dictionary to the desired value of processes that will be executed per instance.

In [142]:
#distributions = {'mpi': {'enabled': True, "processes_per_host": 2}}

distributions = {'mpi': {'enabled': True, 
                         "custom_mpi_options": "-verbose --NCCL_DEBUG=INFO -x OMPI_MCA_btl_vader_single_copy_mechanism=none", 
                         "processes_per_host":8}}

Now, we create the Tensorflow estimator passing the `train_instance_type` and `distribution`

In [143]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12', #1.12
                       distributions=distributions,
                       base_job_name='hvd-mnist-multi-cpu')

tensorflow py2 container will be deprecated soon.


Call `fit()` to start the training

In [144]:
%%time
estimator.fit({"train":s3_train_path, "test":s3_test_path})

2019-06-05 21:22:55 Starting - Starting the training job...
2019-06-05 21:22:59 Starting - Launching requested ML instances.........
2019-06-05 21:24:35 Starting - Preparing the instances for training......
2019-06-05 21:25:54 Downloading - Downloading input data
2019-06-05 21:25:54 Training - Downloading the training image.
2019-06-05 21:26:00,221 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-06-05 21:26:00,227 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-05 21:26:00,667 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-05 21:26:00,677 sagemaker-containers INFO     Starting MPI run as worker node.
2019-06-05 21:26:00,677 sagemaker-containers INFO     Waiting for MPI Master to create SSH daemon.
2019-06-05 21:26:00,679 sagemaker-containers INFO     Cannot connect to host algo-1
2019-06-05 21:26:00,679 sagemaker-containers INFO     Connection failed with exception: 


2019-06-05 21:26:06 Training - Training image download completed. Training in progress.[1,0]<stdout>:Epoch 1/1
[1,5]<stdout>:Epoch 1/1
[1,10]<stdout>:Epoch 1/1
[1,9]<stdout>:Epoch 1/1
[1,12]<stdout>:Epoch 1/1
[1,8]<stdout>:Epoch 1/1
[1,13]<stdout>:Epoch 1/1
[1,15]<stdout>:Epoch 1/1
[1,11]<stdout>:Epoch 1/1
[1,2]<stdout>:Epoch 1/1
[1,14]<stdout>:Epoch 1/1
[1,3]<stdout>:Epoch 1/1
[1,6]<stdout>:Epoch 1/1
[1,1]<stdout>:Epoch 1/1
[1,7]<stdout>:Epoch 1/1
[1,4]<stdout>:Epoch 1/1
[1,5]<stdout>:#015  128/60000 [..............................][1,5]<stdout>: - ETA: 1:00:38 - loss: 2.3149 - acc: 0.1172[1,15]<stdout>:#015  128/60000 [..............................] - ETA: 1:00:37 - loss: 2.3211 - acc: 0.0859[1,12]<stdout>:#015  128/60000 [..............................] - ETA: 1:00:37 - loss: 2.2892 - acc: 0.1094[1,13]<stdout>:#015  128/60000 [..............................] - ETA: 1:00:37 - loss: 2.3034 - acc: 0.0703[1,14]<stdout>:#015  128/60000 [..............................] - ETA: 1:00:30 - 

t>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  768/60000 [..............................] - ETA: 18:38 - loss: 2.3617 - acc: 0.1823[1,4]<stdout>:  768/60000 [..............................][1,4]<stdout>: - ETA: 18:37 - loss: 2.3692 - acc: 0.1862[1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015  768/60000 [..............................] - ETA: 18:40 - loss: 2.3199 - acc: 0.1849[

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 1408/60000 [..............................] - ETA: 14:22 - loss: 2.6908 - acc: 0.2109[1,13]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 1536/60000 [..............................] - ETA: 14:00 - loss: 2.6080 - acc: 0.2227[1,12]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

s: 2.3881 - acc: 0.3022[1,0]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2816/60000 [>.............................] - ETA: 11:57 - loss: 2.2787 - acc: 0.2987[1,1]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010[1,5]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

...

ss: 0.2782 - acc: 0.9160[1,6]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015[1,6]<stdout>:55936/60000 [==========================>...][1,6]<stdout>: - ETA: 42s - loss: 0.2753 - acc: 0.9171[1,1]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01555936/60000 [==========================>...] - ETA: 42s - loss: 0.2747 - acc: 0.9172[1,5]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#0

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01557344/60000 [===========================>..] - ETA: 27s - loss: 0.2703 - acc: 0.9196[1,8]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01557344/60000 [===========================>..] - ETA: 27s - loss: 0.2712 - acc: 0.9194[1,14]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01557344/6

..][1,3]<stdout>: - ETA: 21s - loss: 0.2684 - acc: 0.9185[1,7]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01557984/60000 [===========================>..][1,7]<stdout>: - ETA: 21s - loss: 0.2685 - acc: 0.9199[1,8]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01557984/60000 [===========================>..] - ETA: 21s - loss: 0.2689 - acc: 0.9201[1,1]<stdout>:#010#010#010#010#010#010#010#010#010#01

#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01559392/60000 [============================>.] - ETA: 6s - loss: 0.2665 - acc: 0.9195[1,8]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01559392/60000 [============================>.] - ETA: 6s - loss: 0.2643 - acc: 0.9215[1,10]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#

[1,13]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01560000/60000 [==============================] - 641s 11ms/step - loss: 0.2608 - acc: 0.9209 - val_loss: 0.0348 - val_acc: 0.9901
[1,8]<stdout>:#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#01560000/60000 [==============================] - 642s 11ms/step - loss: 0.2621 - acc: 0.9221 - val_loss: 0.0348 - val_acc: 0.9901
[1,11]<stdout>:#010#010#010#010#010#010#010

## Improving horovod training performance on SageMaker

Performing Horovod training inside a VPC improves the network latency between nodes, leading to higher performance and stability of Horovod training jobs.

For a detailed explanation of how to configure a VPC for SageMaker training, see [Secure Training and Inference with VPC](https://github.com/aws/sagemaker-python-sdk#secure-training-and-inference-with-vpc).

### Setup VPC infrastructure
We will setup following resources as part of VPC stack:
* `VPC`: AWS Virtual private cloud with CIDR block.
* `Subnets`: Two subnets with the CIDR blocks `10.0.0.0/24` and `10.0.1.0/24`
* `Security Group`: Defining the open ingress and egress ports, such as TCP.
* `VpcEndpoint`: S3 Vpc endpoint allowing sagemaker's vpc cluster to dosenload data from S3.
* `Route Table`: Defining routes and is tied to subnets and VPC.

Complete cloud formation template for setting up the VPC stack can be seen [here](./vpc_infra_cfn.json).

In [160]:
print(sagemaker_iam_role)
import boto3
from botocore.exceptions import ClientError
from time import sleep

def create_vpn_infra(stack_name="hvdvpcstack"):
    cfn = boto3.client("cloudformation")

    cfn_template = open("vpc_infra_cfn.json", "r").read()
    
    try:
        vpn_stack = cfn.create_stack(StackName=(stack_name),
                                     TemplateBody=cfn_template)
    except ClientError as e:
        if e.response['Error']['Code'] == 'AlreadyExistsException':
            print("Stack: {} already exists, so skipping stack creation.".format(stack_name))
        else:
            print("Unexpected error: %s" % e)
            raise e

    describe_stack = cfn.describe_stacks(StackName=stack_name)["Stacks"][0]

    while describe_stack["StackStatus"] == "CREATE_IN_PROGRESS":
        describe_stack = cfn.describe_stacks(StackName=stack_name)["Stacks"][0]
        sleep(0.5)

    if describe_stack["StackStatus"] != "CREATE_COMPLETE":
        raise ValueError("Stack creation failed in state: {}".format(describe_stack["StackStatus"]))

    print("Stack: {} created successfully with status: {}".format(stack_name, describe_stack["StackStatus"]))

    subnets = []
    security_groups = []

    for output_field in describe_stack["Outputs"]:

        if output_field["OutputKey"] == "SecurityGroupId":
            security_groups.append(output_field["OutputValue"])
        if output_field["OutputKey"] == "Subnet1Id" or output_field["OutputKey"] == "Subnet2Id":
            subnets.append(output_field["OutputValue"])

    return subnets, security_groups


subnets, security_groups = create_vpn_infra()
print("Subnets: {}".format(subnets))
print("Security Groups: {}".format(security_groups))

arn:aws:iam::328296961357:role/service-role/AmazonSageMaker-ExecutionRole-20190430T072462
Stack: hvdvpcstack created successfully with status: CREATE_COMPLETE
Subnets: ['subnet-06ebce2df1c3f0303', 'subnet-0bdaf2d5c019f21c9']
Security Groups: ['sg-01f003850777a9140']


### VPC training in SageMaker
Now, we create the Tensorflow estimator, passing the `train_instance_type` and `distribution`.

In [161]:
estimator = TensorFlow(entry_point=train_script,
                       role=sagemaker_iam_role,
                       train_instance_count=instance_count,
                       train_instance_type=train_instance_type,
                       script_mode=True,
                       framework_version='1.12',
                       distributions=distributions,
                       security_group_ids=['sg-01f003850777a9140'],
                       subnets=['subnet-06ebce2df1c3f0303', 'subnet-0bdaf2d5c019f21c9'],
                       base_job_name='hvd-mnist-vpc')

tensorflow py2 container will be deprecated soon.


Call `fit()` to start the training

In [ ]:
estimator.fit({"train":s3_train_path, "test":s3_test_path})

2019-06-06 02:37:10 Starting - Starting the training job...
2019-06-06 02:37:15 Starting - Launching requested ML instances......

After training is completed, you can host the saved model by using TensorFlow Serving on SageMaker. For an example that uses TensorFlow Serving, see [(https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_serving_container/tensorflow_serving_container.ipynb](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_serving_container/tensorflow_serving_container.ipynb).

## Reference Links:
* [SageMaker Container MPI Support.](https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_mpi.py)
* [Horovod Official Documentation](https://github.com/uber/horovod)
* [SageMaker Tensorflow script mode example.](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_script_mode_quickstart/tensorflow_script_mode_quickstart.ipynb)